# Cross Validation
- updated 0910
- data  
4th_FE_ver4_before 사용  

1. 교차검증 함수를 만듭니다.  
2. cv = 5 (5 fold validation) 
3. random_state = 77
4. gbm 계열 epoch = 5000, early = 50 으로

>사용함수, mape_exp(), mape_exp_cv(), mape_exp_stac()  
>mape_exp_stac()으로 prediction 값 저장 부탁

## y_predict 값들 피클로 저장해놓기

## ready

In [1]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [ ]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

In [3]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [4]:
# ready
import os
import sys
import joblib
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [5]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [23]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score
import random

## Load Data
4th data 2가지 사용  
data['X'] / data['y']

In [6]:
# load data
## ver4
data_v4 = joblib.load(os.path.join('data', '4th_FE_ver4_before.pk'))
locals().update(data_v4)

In [7]:
X = data_v4['X'] ; y = data_v4['y']
data_v4['X'].shape, data_v4['y'].shape

((35379, 400), (35379,))

In [8]:
X.isnull().sum().sum()

85014

In [27]:
# fillna X
X_full = X.copy().fillna(0)
X_full.isnull().sum().sum()

# dummy data
X_dum = pd.get_dummies(X_full)
print(X_dum.shape)

# log scaling
log_y = np.log(y)
#sns.distplot(log_y)

(35379, 450)


### Cross Validation function

In [11]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [12]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

In [13]:
# MAPE_exp_cv 구현
def MAPE_exp_cv5(model,X,y,cv_splits):        
    # setting
    mape = []
    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=42,shuffle=True)
    for t,v in cv.split(X):
        X_train=X.iloc[t]       # 훈련용
        X_val=X.iloc[v]         # 검증용 분리.
        y_train=y.iloc[t]
        y_val=y.iloc[v]

        # model
        model.fit(X_train,y_train)
        y_pred = model.predict(X_val)
        #y_pred_list.append(np.exp(y_pred))
        # mape
        mape.append(MAPE_exp(y_val, y_pred))
    return mape, np.mean(mape), y_pred_list

In [14]:
# MAPE_exp_cv 구현
def MAPE_exp_cv12(model,X,y,cv_splits):        
    # setting
    mape = []

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,shuffle=False)
    for t,v in cv.split(X):
        X_train=X.iloc[t]       # 훈련용
        X_val=X.iloc[v]         # 검증용 분리.
        y_train=y.iloc[t]
        y_val=y.iloc[v]

        # model
        model.fit(X_train,y_train)
        y_pred = model.predict(X_val)
        # mape
        mape.append(MAPE_exp(y_val, y_pred))
    return mape, np.mean(mape)

In [30]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_cv5_st(model,X,y,cv_splits):        
    # setting
    mape = []
    pred_list = []     # X_val_ 으로 예측
    test_list = []     # X_test 으로 예측
    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        # model
        model.fit(X_train_,y_train_)
        y_pred = model.predict(X_val_)
        pred_list.append(np.exp(y_pred))
        
        test_pred = model.predict(X_test)
        test_list.append(np.exp(test_pred))
        # mape
        mape.append(MAPE_exp(y_val_, y_pred))
    return mape, np.mean(mape), pred_list, test_list 

### Modeling with cv

* lgbm

In [38]:
import lightgbm as lgb

In [ ]:
# lgbm setting
X = X_dum
y = log_y
mape = []
pred_list = []
test_list = []

# split train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

# for 문 돌리기
cv=KFold(n_splits=5,random_state=77,shuffle=True)
for t,v in cv.split(X_train):
    X_train_ = X_train.iloc[t]       # 훈련용
    X_val_ = X_train.iloc[v]         # 검증용 분리.
    y_train_ = y_train.iloc[t]
    y_val_=y.iloc[v]

    # dataset
    train_ds = lgb.Dataset(X_train_, label=y_train_)
    val_ds = lgb.Dataset(X_val_, label=y_val_)

    params={
        'learning_rate':0.01,
        'max_depth':16,
        'boosting':'dart',
        'objective':'regression',
        'metric':'mape',
        'is_training_metric':True,
        'num_leaves':144,
        'feature_fraction':0.9,
        'bagging_fraction':0.7,
        'bagging_freq':5,
        'seed':77
    }

    lgbm_model = lgb.train(params,train_ds,5000,val_ds,verbose_eval=1000)
    y_pred = lgbm_model.predict(X_val_)
    pred_list.append(np.exp(y_pred))
    
    test_pred = lgbm_model.predict(X_test)
    test_list.append(np.exp(test_pred))
    
    # mape
    mape.append(MAPE_exp(y_val_, y_pred))
    
print(mape, np.mean(mape))#, pred_list, test_list 

[1000]	valid_0's mape: 0.147386
[2000]	valid_0's mape: 0.0950816
[3000]	valid_0's mape: 0.0723236
[4000]	valid_0's mape: 0.0673376
[5000]	valid_0's mape: 0.0654671
[1000]	valid_0's mape: 0.147319
[2000]	valid_0's mape: 0.0949697


### CatBoost
예측해야하는 값이 훈련 셋 피처에 들어가버리는 문제, 즉 Data Leakage 문제  
doc : https://catboost.ai/docs/concepts/python-reference_utils_eval_metric.html  
kaggle : https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview
  
    
mape : 31.306861591528538

In [ ]:
!pip install catboost

In [18]:
from catboost import CatBoostRegressor

In [34]:
# catboost setting
cb_model = CatBoostRegressor(iterations=5000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='MAPE',
                             random_seed = 77,
                             bagging_temperature = 0.2,
                             early_stopping_rounds = 500,
                             #group_id = []
                             od_type='Iter',
                             metric_period = 50,     # ?
                             #od_wait=20,
                             thread_count=-1)

In [35]:
print('catboost : \n',MAPE_exp_cv5_st(cb_model,X_dum,log_y,5))

0:	learn: 0.0478830	total: 90.2ms	remaining: 7m 31s
50:	learn: 0.0265858	total: 3.27s	remaining: 5m 17s
100:	learn: 0.0224269	total: 6.41s	remaining: 5m 11s
150:	learn: 0.0203286	total: 9.53s	remaining: 5m 5s
200:	learn: 0.0188922	total: 12.6s	remaining: 5m
250:	learn: 0.0179416	total: 15.7s	remaining: 4m 56s
300:	learn: 0.0171038	total: 18.7s	remaining: 4m 52s
350:	learn: 0.0164104	total: 21.8s	remaining: 4m 49s
400:	learn: 0.0158344	total: 24.9s	remaining: 4m 45s
450:	learn: 0.0152069	total: 27.9s	remaining: 4m 41s
500:	learn: 0.0146919	total: 31s	remaining: 4m 38s
550:	learn: 0.0142364	total: 34.1s	remaining: 4m 35s
600:	learn: 0.0138262	total: 37.1s	remaining: 4m 31s
650:	learn: 0.0134195	total: 40.2s	remaining: 4m 28s
700:	learn: 0.0130553	total: 43.3s	remaining: 4m 25s
750:	learn: 0.0127188	total: 46.4s	remaining: 4m 22s
800:	learn: 0.0123536	total: 49.5s	remaining: 4m 19s
850:	learn: 0.0120448	total: 52.7s	remaining: 4m 16s
900:	learn: 0.0117385	total: 55.8s	remaining: 4m 13s
95

평균 mape 32.73414280484671  
arraty 5개씩 * 2 개 잘나옴

### Class
클래스 돌아가는지 체그 안해봄
- lgbm -> MAPE_Stacking.train(lgb, X, y)
- 그 외 모델 -> MAPE_Stacking.fit(model, X, y)

In [ ]:
class MAPE_Stacking():
    # setting
    mape = []
    pred_list = []     # X_val_ 으로 예측
    test_list = []     # X_test 으로 예측
    
    def __init__(self, model, X, y):
        self.model = model
        self.X = X
        self.y = y
        # 할당
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)

    def MAPE(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

    # xgboost / lgbm 용 (python)
    def train(model, X, y):
        self.X = X_train
        self.y = y_train 
        # for 문 돌리기
        cv=KFold(n_splits=5,random_state=77,shuffle=True)
        for t,v in cv.split(X_train):
            X_train_ = X_train.iloc[t]       # 훈련용
            X_val_ = X_train.iloc[v]         # 검증용 분리.
            y_train_ = y_train.iloc[t]
            y_val_ = y_train.iloc[v]

            # dataset
            train_ds = model.Dataset(X_train_, label=y_train_)
            val_ds = model.Dataset(X_val_, label=y_val_)
            params={
                'learning_rate':0.01,
                'max_depth':16,
                'boosting':'dart',
                'objective':'regression',
                'metric':'mape',
                'is_training_metric':True,
                'num_leaves':144,
                'feature_fraction':0.9,
                'bagging_fraction':0.7,
                'bagging_freq':5,
                'seed':77
            }

            lgbm_model = model.train(params,train_ds,5000,val_ds,verbose_eval=1000)
            y_pred = lgbm_model.predict(X_val_)
            pred_list.append(np.exp(y_pred))
            
            test_pred = lgbm.model.predict(X_test)
            test_list.append(np.exp(test_pred))
            
            # mape
            mape.append(MAPE_exp(y_val_, y_pred))
    
        return mape, np.mean(mape), pred_list, test_list 

    # 그 외 다른 모델용
    def fit(model, X, y):
        self.X = X_train
        self.y = y_train 
        # for 문 돌리기
        cv=KFold(n_splits=5,random_state=77,shuffle=True)
        for t,v in cv.split(X_train):
            X_train_ = X_train.iloc[t]       # 훈련용
            X_val_ = X_train.iloc[v]         # 검증용 분리.
            y_train_ = y_train.iloc[t]
            y_val_ = y_train.iloc[v]

            # model
            model.fit(X_train_,y_train_)
            y_pred = model.predict(X_val_)
            pred_list.append(np.exp(y_pred))
            
            test_pred = model.predict(X_test)
            test_list.append(np.exp(test_pred))
            # mape
            mape.append(MAPE_exp(y_val_, y_pred))
        return mape, np.mean(mape), pred_list, test_list 
